In [1]:
!pip3 install statsmodels
!pip3 install vaderSentiment
!pip3 install nbconvert
!pip3 install tabulate
!pip3 install --upgrade scipy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd

from IPython.display import display
from statistics import mean

from datetime import date, timedelta
import datetime

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.gof import chisquare as chisquare
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from statsmodels.stats.anova import anova_lm

import csv

from tabulate import tabulate

import scipy


In [630]:
current_date = datetime.datetime.strptime("2020-08-22", "%Y-%m-%d").date()

def update_end_date(row, columnName, latestDate):
    if row[columnName] == row[columnName]:
        return row[columnName]
    else:
        return latestDate

def getDays(row, beginColumnName, endColumnName):
    return (row[endColumnName] - row[beginColumnName]).days

def calculate_recency(row, columnName):
    return (current_date - row[columnName]).days

def lookup_index(row, columnName, array):
    if(row[columnName] not in array):
        return -1
    return array.index(row[columnName]) + 1

def fix_signficance(row):
    if('significance' in row['valence']):
        return row['valence']
    else:
        return row['significance']

def fix_valence(row):
    if('significance' in row['significance']):
        return row['valence']
    else:
        return row['significance']  

def compute_sentiment(row):
    post = row['Text']
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(post)
    sentiment = 0
    if (vs["neu"]>0.8):
        sentiment = 0
    elif (vs["pos"]==vs["neg"]):
        sentiment = 0
    elif (vs["pos"]>vs["neg"]):
        sentiment = 1
    elif (vs["neg"]>vs["neu"]):
        sentiment = -1
    return sentiment

def convert_valence_to_sentiment(row):
    valence = row['valence']
    retVal = 0
    if (valence == 'Neither Positive or Negative'):
        retVal = 0
    elif("Positive" in valence):
        retVal = 1
    elif("Negative" in valence):
        retVal = -1
    return retVal

def update_status(row, columnName):
    if row[columnName] == row[columnName]:
        if "ongoing" in row[columnName].lower():
            return "Ongoing"
        return row[columnName]
    else:
        return "Ended"

def update_education_level(row, columnName):
    if 'college' in row[columnName].lower():
        return "College"
    elif 'doctoral' in row[columnName].lower():
        return "Doctoral"
    elif 'master' in row[columnName].lower() or 'grad' in row[columnName].lower():
        return "Graduate"
    elif 'hs' in row[columnName].lower() or 'high school' in row[columnName].lower():
        return "High"
    
    return row[columnName]

def normalize(series):
    return (series - series.mean())/series.std()

In [4]:
control_variables             = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender'
combined_control_variables    = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + data_type'

sr_life_event_variables       = 'Anticipation_sr + LifeEventType_sr + valence_sr + recency_sr + status_sr + Intimacy_sr + Scope_sr + significance_sr'
sm_life_event_variables       = 'Anticipation_sm + LifeEventType_sm + valence_sm + recency_sm + status_sm + Intimacy_sm + Scope_sm + significance_sm'
combined_life_event_variables = 'Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + data_type'

sr_all_variables     = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + significance_label'
sm_all_variables     = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope'
combined_all_variables        = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + data_type'
# combined_sm_sr_all_variables  = 'shipley_vocab_sm + shipley_vocab_sr + shipley_abs_sm + shipley_abs_sr + openness_sm + openness_sr + conscientiousness_sm + conscientiousness_sr + extraversion_sm + extraversion_sr + agreeableness_sm + agreeableness_sr + neuroticism_sm + neuroticism_sr + pos_affect_sm + pos_affect_sr + neg_affect_sm + neg_affect_sr + stai_trait_sm + stai_trait_sr + education_level_sm + education_level_sr + psqi_sm + psqi_sr + age_sm + age_sr + gender_sm + gender_sr + Anticipation_sm + Anticipation_sr + LifeEventFamily_sm + LifeEventFamily_sr + valence_sm + valence_sr + Intimacy_sm + Intimacy_sr + Scope_sm + Scope_sr + data_type'

In [693]:
def load_demographics_data():
    demographics_data = pd.read_csv('data/igtbs_demographics_complete.csv', parse_dates=True)
    demographics_data = demographics_data[['age','gender','snapshot_id', 'shipley.vocab', 'shipley.abs', 'openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism','pos.affect','neg.affect','stai.trait','psqi','educ']]
    demographics_data['education_level'] = demographics_data.apply(update_education_level, columnName='educ', axis=1)
    demographics_data = demographics_data.drop(columns=['educ'])
    demographics_data = demographics_data.rename(columns={
        'shipley.vocab': 'shipley_vocab',
        'shipley.abs': 'shipley_abs',
        'pos.affect': 'pos_affect',
        'neg.affect': 'neg_affect',
        'stai.trait': 'stai_trait'
    })
    demographics_data = demographics_data.dropna()
    return demographics_data

In [681]:
def load_survey_categories():
    df_self_reported_categories = pd.read_csv('data/Life Events Categories Mapping - Self-Reported Categories.csv')
    return df_self_reported_categories

def load_survey_data_without_categories():
    df_survey = pd.read_csv('data/Superimposed/LifeEvents_Curated_non_blinded.csv', parse_dates=True)    
    df_survey = df_survey[['snapshot_id', 'description','UpdatedBeginDate', 'UpdatedEndDate','life_event_type', 'work_perf_impact', 'significance','valence', 'ended_or_ongoing']]

    # Date manipulation
    latest_date = max(datetime.datetime.strptime(str(x), "%Y-%m-%d").date() if x == x else datetime.date.min for x in df_survey['UpdatedEndDate'])
    latest_date = max(latest_date, max(datetime.datetime.strptime(str(x), "%Y-%m-%d").date() if x == x else datetime.date.min for x in df_survey['UpdatedBeginDate']))
    df_survey = df_survey.drop(df_survey[df_survey['UpdatedBeginDate'].isnull() == True].index)

    df_survey['UpdatedBeginDate'] = pd.to_datetime(df_survey['UpdatedBeginDate'], format = '%Y-%m-%d').dt.date
    df_survey['UpdatedEndDate'] = df_survey.apply(update_end_date, columnName='UpdatedEndDate', latestDate=latest_date, axis=1)
    df_survey['UpdatedEndDate'] = pd.to_datetime(df_survey['UpdatedEndDate'], format = '%Y-%m-%d').dt.date

    df_survey['num_of_days'] = df_survey.apply(getDays, endColumnName='UpdatedEndDate', beginColumnName='UpdatedBeginDate', axis=1)
    df_survey['recency'] = df_survey.apply(calculate_recency, columnName='UpdatedEndDate', axis=1)

    # Update values for valence and significance
    df_survey.replace({'valence': {np.nan: 'Neither Positive or Negative'}, 'significance': {np.nan: 'Neither Positive or Negative'}}, inplace=True)
    df_survey['fixed_signficance'] = df_survey.apply(fix_signficance, axis = 1)
    df_survey['fixed_valence'] = df_survey.apply(fix_valence, axis = 1)
    df_survey = df_survey.drop(columns = ['valence', 'significance'])
    df_survey = df_survey.rename(columns={"fixed_signficance": "significance", "fixed_valence": "valence"})
    df_survey['valence'] = df_survey.apply(convert_valence_to_sentiment, axis=1)
    df_survey['ended_or_ongoing'] = df_survey.apply(update_status, columnName='ended_or_ongoing', axis=1)

    # Select columns we are interested in
    df_survey = df_survey[['snapshot_id', 'description', 'UpdatedBeginDate', 'UpdatedEndDate', 'significance', 'valence', 'ended_or_ongoing', 'recency']]

    # Label encoding for significance
    le_significance = LabelEncoder()
    le_significance.fit(df_survey['significance'].values)
    df_survey['significance_label'] = df_survey.apply(lambda x: le_significance.transform([x['significance']])[0], axis=1)
    df_survey = df_survey.drop(columns=['significance'])

    return df_survey

def load_survey_data():
    df_survey_without_categories = load_survey_data_without_categories()
    df_self_reported_categories = load_survey_categories()
    df_survey = pd.merge(df_survey_without_categories, df_self_reported_categories, how="inner", left_on="description", right_on="SR_LifeEvent")
    df_survey = df_survey.drop(columns=['description', 'SR_LifeEvent', 'LifeEventFinal', 'LifeEventFamily2'])
    df_survey['significance_label'] = (df_survey['significance_label'] - df_survey['significance_label'].min()) / (df_survey['significance_label'].max() - df_survey['significance_label'].min())    
    df_survey = df_survey.rename(columns={'ended_or_ongoing':'status', 'LifeEventFamily': 'LifeEventType', 'significance_label': 'significance'})


    return df_survey

In [680]:
def load_social_media_categories():
    df_social_media_categories = pd.read_csv('data/Life Events Categories Mapping - Social Media Categories-2.csv')
    df_social_media_categories['SignificanceRank'] = df_social_media_categories['SignificanceRank'].fillna(df_social_media_categories['SignificanceRank'].max())
    return df_social_media_categories

def load_social_media_data_without_categories():
    df_social_media_data = pd.read_csv('data/Superimposed/Facebook Data For Life Events-Combined - FB Data.csv')
    df_social_media_data = df_social_media_data[['snapshot_id', 'created_time', 'Text', 'final_life_event_category_2', 'ended/ongoing']]
    df_social_media_data = df_social_media_data.replace({'PostiveMove':'PositiveMove', 'Negative Move':'NegativeMove'})
    df_social_media_data = df_social_media_data.drop(df_social_media_data[((df_social_media_data['final_life_event_category_2'].isnull() == True))].index)
    df_social_media_data['created_date'] = pd.to_datetime(df_social_media_data['created_time'], format = '%Y-%m-%d %H:%M:%S').dt.date
    df_social_media_data['valence'] = df_social_media_data.apply(compute_sentiment, axis=1)
    df_social_media_data = df_social_media_data.drop(columns=['created_time','Text'])
    df_social_media_data['recency'] = df_social_media_data.apply(calculate_recency, columnName='created_date', axis=1)
    df_social_media_data['ended/ongoing'] = df_social_media_data.apply(update_status, columnName='ended/ongoing', axis=1)
    return df_social_media_data

def load_social_media_data():
    df_social_media_data = load_social_media_data_without_categories()
    df_social_media_categories = load_social_media_categories()
    df_social_media_data_with_categories = pd.merge(df_social_media_data, df_social_media_categories, how="inner", left_on='final_life_event_category_2', right_on='SM_LifeEvent')
    df_social_media_data_with_categories = df_social_media_data_with_categories.drop(columns=['final_life_event_category_2','SM_LifeEvent','LifeEventFamily2','Comments'])
    df_social_media_data_with_categories = df_social_media_data_with_categories.rename(columns={'ended/ongoing':'status', 'LifeEventFamily': 'LifeEventType', 'SignificanceRank': 'significance'})
    df_social_media_data_with_categories['significance'].fillna(df_social_media_data_with_categories['significance'].max())
    df_social_media_data_with_categories['significance'] = (df_social_media_data_with_categories['significance'] - df_social_media_data_with_categories['significance'].min()) / (df_social_media_data_with_categories['significance'].max() - df_social_media_data_with_categories['significance'].min())    
    
    return df_social_media_data_with_categories

In [8]:
def load_dailies_data():
    df_dailies = pd.read_csv('data/Superimposed/dailies_scores.csv', low_memory=False)
    df_dailies = df_dailies[['snapshot_id','day', 'alc_status', 'alc.quantity.d', 'anxiety.d', 'pos.affect.d', 'neg.affect.d','sleep.d', 'stress.d']]
    df_dailies['day'] = pd.to_datetime(df_dailies['day'], format='%Y-%m-%d').dt.date
    df_dailies = df_dailies.rename(columns={'alc.quantity.d': 'alc_quantity',
    'anxiety.d': 'anxiety',
    'pos.affect.d': 'pos_affect',
    'neg.affect.d': 'neg_affect',
    'sleep.d': 'sleep',
    'stress.d': 'stress'})
    df_dailies['sleep'] = df_dailies['sleep'] + 1
    return df_dailies

In [557]:
min(load_dailies_data()['sleep'])

1.0

In [9]:
def calculate_year(row, columnName):
    return row[columnName].isocalendar()[0]

def calculate_week(row, columnName):
    return row[columnName].isocalendar()[1]

In [740]:
def build_stacked_df(dependent_variable):
    try:
        merged_data = pd.read_csv('Linear Regression/Calendar Week/all_data_stacked_'+dependent_variable+'.csv')
    except:
        df_dailies = load_dailies_data()
        df_social_media = load_social_media_data()
        df_survey = load_survey_data()
        df_demographics = load_demographics_data()

        df_social_media['year'] = df_social_media.apply(calculate_year, columnName='created_date', axis=1)
        df_social_media['week'] = df_social_media.apply(calculate_week, columnName='created_date', axis=1)
        df_social_media['type'] = 'Social Media'

        df_survey['year'] = -1
        df_survey['week'] = -1
        survey_rows = []
        for i, row in df_survey.iterrows():
            s_date = row['UpdatedBeginDate']
            e_date = row['UpdatedEndDate']
            delta = timedelta(days=7)

            while s_date <= e_date:
                r = row.copy(deep=True)
                r['year'] = s_date.isocalendar()[0]
                r['week'] = s_date.isocalendar()[1]
                r['recency'] = (current_date - s_date).days
                survey_rows.append(r.values)
                s_date +=delta
        df_survey = pd.DataFrame(survey_rows, columns=df_survey.columns)
        df_survey['type'] = 'Survey'

        df_dailies['year'] = df_dailies.apply(calculate_year, columnName='day', axis=1)
        df_dailies['week'] = df_dailies.apply(calculate_week, columnName='day', axis=1)
        df_dailies = df_dailies[['snapshot_id', dependent_variable, 'year', 'week']]
        df_dailies = df_dailies.groupby(['snapshot_id', 'year', 'week']).mean().reset_index()

        merged_data = pd.merge(df_dailies, df_demographics, how="inner", on="snapshot_id")
        merged_data_sm = pd.merge(merged_data, df_social_media, how="inner", on=["snapshot_id", "year", "week"])
        merged_data_sr = pd.merge(merged_data, df_survey, how="inner", on=["snapshot_id", "year", "week"])

        merged_data = pd.concat([merged_data_sm,merged_data_sr],ignore_index=True)
        
        merged_data['recency'] = (merged_data['recency'] - merged_data['recency'].min()) / (merged_data['recency'].max() - merged_data['recency'].min())    
        merged_data.to_csv('Linear Regression/Calendar Week/all_data_stacked_'+dependent_variable+'.csv', index=False)
        
    merged_data = merged_data.drop(columns=['created_date', 'UpdatedBeginDate', 'UpdatedEndDate'])
    return merged_data

In [745]:
def build_merged_df(dependent_variable):
    try:
        merged_data = pd.read_csv('Linear Regression/Calendar Week/all_data_'+dependent_variable+'.csv')
    except:
        df_dailies = load_dailies_data()
        df_social_media = load_social_media_data()
        df_survey = load_survey_data()
        df_demographics = load_demographics_data()
        
        df_social_media['year'] = df_social_media.apply(calculate_year, columnName='created_date', axis=1)
        df_social_media['week'] = df_social_media.apply(calculate_week, columnName='created_date', axis=1)
        df_social_media.rename(columns=lambda x: x+'_sm', inplace=True)
        df_social_media = df_social_media.rename(columns={
            'snapshot_id_sm': 'snapshot_id',
            'week_sm' : 'week',
            'year_sm' : 'year'
        })

        df_survey['year'] = -1
        df_survey['week'] = -1
        survey_rows = []
        for i, row in df_survey.iterrows():
            s_date = row['UpdatedBeginDate']
            e_date = row['UpdatedEndDate']
            delta = timedelta(days=7)

            while s_date <= e_date:
                r = row.copy(deep=True)
                r['year'] = s_date.isocalendar()[0]
                r['week'] = s_date.isocalendar()[1]
                r['recency'] = (current_date - s_date).days

                survey_rows.append(r.values)
                s_date +=delta
        df_survey = pd.DataFrame(survey_rows, columns=df_survey.columns)
        df_survey.rename(columns=lambda x: x+'_sr', inplace=True)
        df_survey = df_survey.rename(columns={
            'snapshot_id_sr': 'snapshot_id',
            'week_sr' : 'week',
            'year_sr' : 'year'
        })

        df_dailies['year'] = df_dailies.apply(calculate_year, columnName='day', axis=1)
        df_dailies['week'] = df_dailies.apply(calculate_week, columnName='day', axis=1)
        df_dailies = df_dailies[['snapshot_id', dependent_variable, 'year', 'week']]
        df_dailies = df_dailies.groupby(['snapshot_id', 'year', 'week']).mean().reset_index()

        df_social_media['recency_sm'] = (df_social_media['recency_sm'] - df_social_media['recency_sm'].min()) / (df_social_media['recency_sm'].max() - df_social_media['recency_sm'].min())    
        df_survey['recency_sr'] = (df_survey['recency_sr'] - df_survey['recency_sr'].min()) / (df_survey['recency_sr'].max() - df_survey['recency_sr'].min())    
        
        merged_data = pd.merge(df_dailies, df_demographics, how="inner", on="snapshot_id")
        merged_data = pd.merge(merged_data, df_social_media, how="left", on=["snapshot_id", "year", "week"])
        merged_data = pd.merge(merged_data, df_survey, how="left", on=["snapshot_id", "year", "week"])
        

        merged_data.to_csv('Linear Regression/Calendar Week/all_data_'+dependent_variable+'.csv', index=False)
        
    merged_data = merged_data.drop(columns=['created_date_sm', 'UpdatedBeginDate_sr', 'UpdatedEndDate_sr'])
    return merged_data

## Run Regression  

In [597]:
control_variables = ['shipley_vocab', 'shipley_abs', 'openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism', 'pos_affect', 'neg_affect', 'stai_trait', 'education_level', 'psqi', 'age', 'gender']
sr_life_event_variables = ['Anticipation_sr', 'LifeEventType_sr', 'valence_sr', 'recency_sr', 'status_sr', 'Intimacy_sr', 'Scope_sr', 'significance_sr']
sm_life_event_variables = ['Anticipation_sm', 'LifeEventType_sm', 'valence_sm', 'recency_sm', 'status_sm', 'Intimacy_sm', 'Scope_sm', 'significance_sm']
life_event_variables = ['Anticipation', 'LifeEventType', 'valence', 'recency', 'status', 'Intimacy', 'Scope', 'significance']

In [752]:
def get_summary(data, formula):
    model = smf.ols(formula=formula, data = data).fit()

    return model.summary()

In [754]:
def get_model(data, formula):
    model = smf.ols(formula=formula, data = data).fit()

    return model

In [822]:
def generate_summary_file(data, formula, filename):
    t = get_summary(data, formula).as_text()

    with open('Linear Regression/Calendar Week/summary/'+filename+'.txt', 'w') as the_file:
        the_file.write(t)

In [839]:
def generate_summary_files(dependent_variable):
    data = build_merged_df(dependent_variable)
    all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
    all_data = all_data[all_data[dependent_variable].notnull()]

    generate_summary_file(all_data, dependent_variable+'~'+' + '.join(control_variables), 'all_demo_trait_'+dependent_variable)
    generate_summary_file(all_data, dependent_variable+'~'+' + '.join(control_variables+sr_life_event_variables), 'all_sm_sr_'+dependent_variable)
    generate_summary_file(all_data, dependent_variable+'~'+' + '.join(control_variables+sr_life_event_variables+sm_life_event_variables), 'all_sm_sr_demo_trait_'+dependent_variable)
    all_variables = ' + '.join(control_variables+sr_life_event_variables+sm_life_event_variables)
    combined_control = ' + '.join(control_variables)
    combined_life_event = ' + '.join(sr_life_event_variables+sm_life_event_variables)
    final_var = '('+all_variables+')**2 - ' + ' ('+combined_control+')**2 - ' + ' ('+combined_life_event+')**2' + ' + ' + all_variables

    generate_summary_file(all_data, ModelDesc.from_formula(dependent_variable+'~'+final_var).describe(), 'all_sm_sr_demo_trait_interaction_'+dependent_variable)

    data = build_stacked_df(dependent_variable)
    generate_summary_file(data, dependent_variable+'~'+' + '.join(control_variables), 'stacked_demo_trait_'+dependent_variable)
    generate_summary_file(data, dependent_variable+'~'+' + '.join(control_variables+life_event_variables), 'stacked_sm_sr_demo_trait'+dependent_variable)
    all_variables = ' + '.join(control_variables + life_event_variables)
    combined_control = ' + '.join(control_variables)
    combined_life_event = ' + '.join(life_event_variables)
    final_var = '('+all_variables+')**2 - ' + ' ('+combined_control+')**2 - ' + ' ('+combined_life_event+')**2' + ' + ' + all_variables

    generate_summary_file(data, ModelDesc.from_formula(dependent_variable+'~'+final_var).describe(), 'stacked_sm_sr_demo_trait_interaction_'+dependent_variable)
    

In [836]:
ModelDesc.from_formula('a~(b + c + d + e)**2 - (c + d)**2 + d')

ModelDesc(lhs_termlist=[Term([EvalFactor('a')])],
          rhs_termlist=[Term([]),
                        Term([EvalFactor('b')]),
                        Term([EvalFactor('e')]),
                        Term([EvalFactor('b'), EvalFactor('c')]),
                        Term([EvalFactor('b'), EvalFactor('d')]),
                        Term([EvalFactor('b'), EvalFactor('e')]),
                        Term([EvalFactor('c'), EvalFactor('e')]),
                        Term([EvalFactor('d'), EvalFactor('e')]),
                        Term([EvalFactor('d')])])

In [840]:
generate_summary_files('stress')

In [841]:
generate_summary_files('sleep')

In [842]:
generate_summary_files('anxiety')

In [722]:
def run_regression(data, dependent_variable, variables):

    control_data = data[variables['control']+[dependent_variable]].dropna()
    train_data, test_data = train_test_split(control_data, test_size=0.20, random_state=805)
    f_exp = test_data[dependent_variable].values

    baseline_formula = dependent_variable+'~'+' + '.join(variables['control'])
    baseline = smf.ols(formula=baseline_formula,
                  data = control_data).fit()
    baseline_train = smf.ols(formula=baseline_formula,
                  data = train_data).fit()
    f_obs = baseline_train.predict(test_data).values
    pearson_baseline = scipy.stats.pearsonr(f_obs, f_exp)
    
    
    life_events_data = data[variables['life_events']+[dependent_variable]].dropna()    
    train_data, test_data = train_test_split(life_events_data, test_size=0.20, random_state=805)
    f_exp = test_data[dependent_variable].values

    life_events_formula = dependent_variable+'~'+' + '.join(variables['life_events'])
    life_events = smf.ols(formula=life_events_formula, data = life_events_data).fit() 
    life_events_train = smf.ols(formula=life_events_formula, data = train_data).fit()     
    f_obs = life_events_train.predict(test_data).values
    pearson_life_events = scipy.stats.pearsonr(f_obs, f_exp)    

    
    all_variable_data = data[variables['all_variables']+[dependent_variable]].dropna()
    train_data, test_data = train_test_split(all_variable_data, test_size=0.20, random_state=805)
    f_exp = test_data[dependent_variable].values    
    
    all_variables_formula = dependent_variable+'~'+' + '.join(variables['all_variables'])
    all_variables = smf.ols(formula=all_variables_formula, data = all_variable_data).fit() 
    all_variables_train = smf.ols(formula=all_variables_formula, data = train_data).fit() 
    f_obs = all_variables_train.predict(test_data).values
    pearson_all_variables = scipy.stats.pearsonr(f_obs, f_exp)    

    print("Baseline R-square adj", baseline.rsquared_adj)
    print("Life Events R-square adj", life_events.rsquared_adj)
    print("All Variables R-square adj", all_variables.rsquared_adj)
    print()
    print(anova_lm(baseline, all_variables))
    print(anova_lm(life_events, all_variables))
    print()
    print("Baseline pearson", pearson_baseline)
    print("Life Events pearson", pearson_life_events)
    print("All Variables pearson", pearson_all_variables)    
    return all_variables.rsquared_adj

In [723]:
def run_all_regression(dependent_variable):
    data = build_merged_df(dependent_variable)

    social_media_data = data[data['LifeEventType_sm'].notnull()]
    social_media_data = social_media_data[social_media_data[dependent_variable].notnull()]

    social_media_variables = {
        'control': control_variables,
        'life_events': sm_life_event_variables,
        'all_variables': sm_life_event_variables + control_variables
    }

    survey_data = data[data['LifeEventType_sr'].notnull()]
    survey_data = survey_data[survey_data[dependent_variable].notnull()]
    survey_variables = {
        'control': control_variables,
        'life_events': sr_life_event_variables,
        'all_variables': sr_life_event_variables + control_variables
    }
    
    all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
    all_data = all_data[all_data[dependent_variable].notnull()]
    all_data_variables = {
        'control': control_variables,
        'life_events': sm_life_event_variables + sr_life_event_variables,
        'all_variables': sm_life_event_variables + sr_life_event_variables + control_variables
    }
    
    print("---------SOCIAL MEDIA---------")
    run_regression(social_media_data, dependent_variable, social_media_variables)
    print()
    
    print("---------SURVEY---------")
    run_regression(survey_data, dependent_variable, survey_variables)
    print()
    
    print("---------SOCIAL MEDIA + SURVEY---------")
    run_regression(all_data, dependent_variable, all_data_variables)
    print()    

In [726]:
def run_stacked_regression(dependent_variable):
    data = build_stacked_df(dependent_variable)

    all_data = data[data[dependent_variable].notnull()]
    all_data_variables = {
        'control': control_variables,
        'life_events': life_event_variables,
        'all_variables': life_event_variables + control_variables
    }
    
    print("---------SOCIAL MEDIA + SURVEY---------")
    run_regression(all_data, dependent_variable, all_data_variables)
    print()    

### Stress 

In [786]:
run_all_regression('stress')

---------SOCIAL MEDIA---------
Baseline R-square adj 0.10907194727705449
Life Events R-square adj 0.05937230101079394
All Variables R-square adj 0.16120339770359016

   df_resid         ssr  df_diff    ss_diff         F        Pr(>F)
0     722.0  340.106908      0.0        NaN       NaN           NaN
1     710.0  314.884029     12.0  25.222879  4.739376  1.742722e-07
   df_resid         ssr  df_diff    ss_diff         F        Pr(>F)
0     726.0  361.068836      0.0        NaN       NaN           NaN
1     710.0  314.884029     16.0  46.184807  6.508589  7.791949e-14

Baseline pearson (0.26822579146241304, 0.0009805374505623466)
Life Events pearson (0.20493718617723544, 0.012468220369030815)
All Variables pearson (0.34620702903789474, 1.6315319295293034e-05)

---------SURVEY---------
Baseline R-square adj 0.14474634444517942
Life Events R-square adj 0.01890986527088767
All Variables R-square adj 0.1614982383663217

   df_resid         ssr  df_diff    ss_diff         F    Pr(>F)
0    12

In [787]:
run_stacked_regression('stress')

---------SOCIAL MEDIA + SURVEY---------
Baseline R-square adj 0.13179997436452362
Life Events R-square adj 0.03450161250160477
All Variables R-square adj 0.1612594045619733

   df_resid         ssr  df_diff    ss_diff         F        Pr(>F)
0    1843.0  785.435389      0.0        NaN       NaN           NaN
1    1830.0  753.432047     13.0  32.003342  5.979419  5.320741e-11
   df_resid         ssr  df_diff     ss_diff          F        Pr(>F)
0    1846.0  874.880195      0.0         NaN        NaN           NaN
1    1830.0  753.432047     16.0  121.448147  18.436476  4.690228e-49

Baseline pearson (0.3154235291469514, 4.884945259033782e-10)
Life Events pearson (0.13697400142439178, 0.008158290043142755)
All Variables pearson (0.33322395304081726, 4.258826568108379e-11)



In [788]:
run_all_regression('sleep')

---------SOCIAL MEDIA---------
Baseline R-square adj 0.13044553100333411
Life Events R-square adj 0.018523183700833945
All Variables R-square adj 0.13803662946251982

   df_resid          ssr  df_diff    ss_diff         F    Pr(>F)
0     653.0  1046.111369      0.0        NaN       NaN       NaN
1     641.0  1017.922677     12.0  28.188692  1.479234  0.126886
   df_resid          ssr  df_diff     ss_diff         F        Pr(>F)
0     657.0  1187.991585      0.0         NaN       NaN           NaN
1     641.0  1017.922677     16.0  170.068908  6.693422  3.318120e-14

Baseline pearson (0.393488744284707, 2.5579854045044186e-06)
Life Events pearson (0.0727072481263923, 0.40379211085773664)
All Variables pearson (0.3351792277116428, 7.533688434925699e-05)

---------SURVEY---------
Baseline R-square adj 0.08164183887495735
Life Events R-square adj 0.015054833343084995
All Variables R-square adj 0.11315026643077875

   df_resid          ssr  df_diff    ss_diff         F        Pr(>F)
0    11

In [789]:
run_stacked_regression('sleep')

---------SOCIAL MEDIA + SURVEY---------
Baseline R-square adj 0.06676800727341381
Life Events R-square adj 0.010005883226233636
All Variables R-square adj 0.07828724982585633

   df_resid          ssr  df_diff    ss_diff         F    Pr(>F)
0    1750.0  2581.683798      0.0        NaN       NaN       NaN
1    1737.0  2530.875581     13.0  50.808218  2.682376  0.000963
   df_resid          ssr  df_diff    ss_diff         F        Pr(>F)
0    1753.0  2743.404911      0.0        NaN       NaN           NaN
1    1737.0  2530.875581     16.0  212.52933  9.116495  5.442639e-22

Baseline pearson (0.28215034764799257, 6.665717665069504e-08)
Life Events pearson (0.0905279642863686, 0.08899125194419409)
All Variables pearson (0.3009376370943917, 7.625587865399592e-09)



In [790]:
run_all_regression('anxiety')

---------SOCIAL MEDIA---------
Baseline R-square adj 0.1675543210113799
Life Events R-square adj 0.05759560455052348
All Variables R-square adj 0.20597707056754688

   df_resid         ssr  df_diff    ss_diff         F    Pr(>F)
0     722.0  242.774207      0.0        NaN       NaN       NaN
1     710.0  227.719822     12.0  15.054385  3.911463  0.000008
   df_resid         ssr  df_diff    ss_diff         F        Pr(>F)
0     726.0  276.365207      0.0        NaN       NaN           NaN
1     710.0  227.719822     16.0  48.645385  9.479363  1.241943e-21

Baseline pearson (0.35812996941414127, 7.857072126800395e-06)
Life Events pearson (0.14166457652616188, 0.08589077676005877)
All Variables pearson (0.3569311324607578, 8.46740774095825e-06)

---------SURVEY---------
Baseline R-square adj 0.2161289450567555
Life Events R-square adj 0.03379977071500295
All Variables R-square adj 0.232510784216577

   df_resid         ssr  df_diff    ss_diff         F    Pr(>F)
0    1253.0  447.859244   

In [791]:
run_stacked_regression('anxiety')

---------SOCIAL MEDIA + SURVEY---------
Baseline R-square adj 0.19934215456241433
Life Events R-square adj 0.061474698726827315
All Variables R-square adj 0.23386344235575773

   df_resid         ssr  df_diff    ss_diff         F        Pr(>F)
0    1843.0  611.717138      0.0        NaN       NaN           NaN
1    1830.0  581.213409     13.0  30.503729  7.387969  2.510367e-14
   df_resid         ssr  df_diff     ss_diff          F        Pr(>F)
0    1846.0  718.217579      0.0         NaN        NaN           NaN
1    1830.0  581.213409     16.0  137.004171  26.960582  8.150553e-73

Baseline pearson (0.38164032800144354, 2.409808294259408e-14)
Life Events pearson (0.2352883428500234, 4.488739267686155e-06)
All Variables pearson (0.42251658058842917, 1.5378152679254838e-17)

